In [2]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.3/448.3 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00


In [3]:
import requests
import time
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup

# Sending GET request
website = requests.get("http://lhr.nu.edu.pk/faculty/")
html = website.content
# Using Beautiful Soup Object
soup = BeautifulSoup(html, "html.parser")
soup.prettify

# Getting information of each department
cs = soup.find("div", class_ = "container", id = "fsc")
ee = soup.find("div", class_ = "container", id = "ee")
cv = soup.find("div", class_ = "container", id = "cv")
fsm = soup.find("div", class_ = "container", id = "fsm")
ss = soup.find("div", class_ = "container", id = "ss")

all_departments = [cs,ee,cv,fsm,ss]

##### Lahore Faculty

In [ ]:
df_lahore = pd.DataFrame()
for department in all_departments:

    department_name = cs.find('h1').get_text()
    all_faculty = department.find_all("div",class_="facultyCard")

    for faculty in all_faculty:
        # getting faculty name
        name = faculty.find('h5').get_text()
        # getting faculty designation
        designation = faculty.find('p').get_text()
        # checking if faculty is HEC approved
        hec = True if designation.find("HEC Approved PhD Supervisor") != -1 else False
        # formatting faculty designation
        designation = designation.split("\n")[0]
        # getting faculty email address
        email = faculty.find('p',class_="mb-0 text-center").get_text()
        # faculty profile page URL
        link = "http://lhr.nu.edu.pk" + faculty.find('a')['href']
        # Extracting ID from URL
        id = link.split("/")[-1]
        # Getting image URL
        img = "http://lhr.nu.edu.pk" + faculty.find('img')['src']

        # Creating BeautifulSoup Object for faculty page
        profile_page = requests.get(link)
        profile_page_content = profile_page.content
        soup2 = BeautifulSoup(profile_page_content, "html.parser")

        # Getting phone number extension
        extension = soup2.find("span", class_ = 'small').get_text().split(" ")[-1].split(":")[-1]
        # Trying to get 1st element of education info (this will be highest education level as arranged in descending order)
        try:
            highest_edu = soup2.find("div",class_ = 'container').find('li').get_text().strip(" ")
        except:
            highest_edu = "None"

        # print('--------------------------------------------------------------------------------------------------------------------')
        # print(id)
        # print(name)
        # print(email)
        # print(designation)
        # print(department)
        # print(highest_edu)
        # print(link)
        # print(img)
        # print(hec)
        # print(extension)

        info =  {
            'ID': [id],
            'Name': [name],
            'Designation': [designation],
            'HEC Approved PHD Supervisor': [hec],
            'Highest Education': [highest_edu],
            'Email': [email],
            'Department': [department_name],
            'Extension': [extension],
            'ImageURL': [img]
        }
        # inserting new row into DataFrame
        row = pd.DataFrame(info)
        df_lahore = pd.concat([df_lahore,row],ignore_index=True)

In [ ]:
# saving DataFrame into a .csv file
df_lahore.to_csv('lhr.csv')

#### Islamabad Faculty

In [ ]:
# test = requests.get("http://isb.nu.edu.pk/Faculty/allfaculty/")
driver = webdriver.Chrome()
driver.get("http://isb.nu.edu.pk/Faculty/allfaculty/")

time.sleep(4)

# html = test.content
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')
# soup.prettify

In [ ]:
df_islamabad = pd.DataFrame()
#for all depts
all_departments = soup.find_all("div", class_ = "row blog")

for department in all_departments:
    department_name = department.find('div',class_='animated fadeInLeft').get_text() + " " + department.find('div',class_='animated fadeInRight').get_text()
    all_faculty = department.find_all('div',class_='our-team')

    for faculty in all_faculty:
        #getting name,designation,link,id
        name = faculty.find('h3',class_='title').get_text()
        designation = faculty.find('span',class_='post').get_text()
        email = faculty.find('p').get_text()
        link = "http://isb.nu.edu.pk" + faculty.find('a')['href']
        id = faculty.find('div',class_='pic').find('img')['id']

        # extracting id from string 'image1234' id = 1234
        new_id = ''
        for char in id:
            if char.isdigit():
                new_id = new_id + char
        id = new_id


        img = "http://isb.nu.edu.pk/Images/Profile/CS/" + faculty.find('div',class_='pic').find('img')['src'].split('/')[-1]

        faculty_page = webdriver.Chrome()
        faculty_page.get(link)

        time.sleep(4)

        html2 = faculty_page.page_source
        soup2 = BeautifulSoup(html2,'html.parser')
        #checking for hec approved or not
        try:
            if soup2.find('label', style='font-size:9pt;').get_text().find('HEC Approved PhD Supervisor') != -1:
                hec = 'Approved'
        except:
            hec = 'Unapproved'

        extension = soup2.find('div',class_='profile-info').find_all('p')[-1].get_text().split(' ')[-1].split('\n')[0]
        try:
            highest_edu = soup2.find('span',id='DegreeName').find_all('label')[0].get_text()
        except:
            highest_edu = 'unavailable'

        # print('--------------------------------------------------------------------------------------------------------------------')
        # print(id)
        # print(name)
        # print(email)
        # print(designation)
        # print(department)
        # print(highest_edu)
        # print(link)
        # print(img)
        # print(hec)
        # print(extension)

        info =  {
            'ID': [id],
            'Name': [name],
            'Designation': [designation],
            'HEC Approved PHD Supervisor': [hec],
            'Highest Education': [highest_edu],
            'Email': [email],
            'Department': [department_name],
            'Extension': [extension],
            'ImageURL': [img]
        }
        # inserting new row into DataFrame
        row = pd.DataFrame(info)
        df_islamabad = pd.concat([df_islamabad,row],ignore_index=True)

In [ ]:
df_islamabad.to_csv('isb.csv')

#### Karachi Campus

In [ ]:
url_khi = "https://khi.nu.edu.pk/"
page = requests.get(url_khi)
htmlContent = page.content
#print(htmlContent)
soup = BeautifulSoup(htmlContent, "html.parser")
# print(soup.prettify)

cs = soup.find('li',class_="menu-item menu-item-type-post_type menu-item-object-page menu-item-6495")
cy = soup.find('li',class_="menu-item menu-item-type-post_type menu-item-object-page menu-item-6773")
ai = soup.find('li',class_="menu-item menu-item-type-post_type menu-item-object-page menu-item-6791")
se = soup.find('li',class_="menu-item menu-item-type-post_type menu-item-object-page menu-item-6790")
sh = soup.find('li',class_="menu-item menu-item-type-post_type menu-item-object-page menu-item-7414")
ee = soup.find('li',class_="menu-item menu-item-type-post_type menu-item-object-page menu-item-6825")
ms = soup.find('li',class_="menu-item menu-item-type-post_type menu-item-object-page menu-item-6824")

all_departments_links = [cs,cy,ai,se,sh,ee,ms]

In [ ]:
df_karachi = pd.DataFrame()
for department in all_departments_links:
    # Getting Department Name
    department_name = department.get_text()
    # Getting Department Page Link
    department_page_link = department.find('a')['href']
    # print(link)

    # Sending GET request to Department Page
    department_page = requests.get(department_page_link)
    # Extracting html content
    htmlContent2 = department_page.content
    #print(htmlContent2)
    # Cretating Beautiful Soup Object
    soup2 = BeautifulSoup(htmlContent2, "html.parser")
    # print(soup2.prettify)

    # Getting image div to extract info
    image_block = soup2.find_all('div', class_="gdlr-core-personnel-list-image")
    # Getting content div to extract info
    info_block = soup2.find_all('div', class_="gdlr-core-personnel-list-content-wrap")
    size = len(info_block)

    for i in range(size):
        # Extracting faculty Name
        name = info_block[i].find('h3').get_text().split(',')[0].strip()
        # Extracting Email
        email = info_block[i].find('div',class_='kingster-personnel-info-list kingster-type-email').get_text()

        # Trying to fetch extension as integer
        try:
            extension = int(info_block[i].find('div',class_='kingster-personnel-info-list kingster-type-phone').get_text())
        except:
            extension = -1

        # Getting profile page link
        profile_page = info_block[i].find('a')['href']

        # Extracting faculty designation
        designation = info_block[i].find('div',class_='gdlr-core-personnel-list-position gdlr-core-info-font gdlr-core-skin-caption').get_text()

        # Extracting image from image div
        try:
            img = image_block[i].find('img')['src']
        except:
            img = 'unavailable'

        # Sending GET request to faculty profile page
        faculty_profile_page = requests.get(profile_page)
        # Extracting html content
        htmlContent3 = faculty_profile_page.content
        # Beautiful Soup Object
        soup3 = BeautifulSoup(htmlContent3, "html.parser")
        #print(htmlContent2)

        # Extracting Highest Education
        try:
            highest_edu = soup3.find('span',class_='gdlr-core-icon-list-content').get_text()
        except:
            # Some faculty members have education in different class
            highest_edu = soup3.find('span',class_='gdlr-core-title-item-caption gdlr-core-info-font gdlr-core-skin-caption').get_text()


        # Checking if faculty member is HEC approved
        hec = True if soup3.find('span',class_='gdlr-core-title-item-caption gdlr-core-info-font gdlr-core-skin-caption').get_text().find('HEC Approved PhD Supervisor') != -1 else False

        # Extracting faculty id from script tag
        faculty_id = soup3.find('script',id='burst-js-extra').get_text().split(',')[1].split(':')[-1].strip('\""')

        # ? Placing all information into a dictionary
        row =  {
            'ID': faculty_id,
            'Name': name,
            'Designation': designation,
            'HEC Approved PHD Supervisor': hec,
            'Highest Education': highest_edu,
            'Email': email,
            'Department': department,
            'Extension': extension,
            'ImageURL': img
        }

        df_karachi = df_karachi.append(row,ignore_index=True)
df_karachi.to_csv('khi.csv')

C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.append(row,ignore_index=True)
C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.append(row,ignore_index=True)
C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.append(row,ignore_index=True)
C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.ap

C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.append(row,ignore_index=True)
C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.append(row,ignore_index=True)
C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.append(row,ignore_index=True)
C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.ap

C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.append(row,ignore_index=True)
C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.append(row,ignore_index=True)
C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.append(row,ignore_index=True)
C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.ap

C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.append(row,ignore_index=True)
C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.append(row,ignore_index=True)
C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.append(row,ignore_index=True)
C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.ap

C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.append(row,ignore_index=True)
C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.append(row,ignore_index=True)
C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.append(row,ignore_index=True)
C:\Users\legio\AppData\Local\Temp\ipykernel_21476\3400696551.py:83: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_karachi = df_karachi.ap

In [ ]:
df_karachi.to_csv('khi.csv')

#### Concat All DataFrames

In [ ]:
df = pd.concat([df_lahore,df_islamabad,df_karachi],axis=0,ignore_index=True)
df.to_csv('fast_faculty.csv')

faculty = pd.read_csv('fast_faculty.csv')

sample_ = faculty.sample(frac=0.7)
sample_.to_csv('sample.csv')